# 신경망 모델 구성하기

신경망은 데이터에 대한 연산을 수행하는 layer(계층)/module로 구성.

torch.nn 네임 스페이스는 신경망을 구성하는데 필요한 모든 구성요소를 제공

PyTorch의 모든 모듈은 nn.Module의 subclass, 신경망은 다른 모듈(계층; layer)로 구성된 모듈

FashionMNIST 데이터셋의 이미지들을 분류하는 신경망 구성

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기

torch.cuda 또는 torch.backends.mps가 사용 가능한지 확인, 그렇지 않으면 CPU 사용

In [2]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'

)

print(f'using {device} device')

using cuda device


## 클래스 정의하기

신경망 모델을 nn.Module의 하위 클래스로 정의하고, __init__에서 신경망 계층을 초기화
nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512), 
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    


In [4]:
# NeuralNetwork의 인스턴스를 생성하고, 이를 device로 이동한 뒤 구조를 출력

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델을 사용하기 위해 입력 데이터를 전달, 일부 백그라운드 연산들과 함께 모델의 forward를 실행

model.forward()를 직접 호출하지 말것!!

모델에 입력을 전달하여 호출하면 2차원 텐서를 반환
dim=0은 각 class에 대한 raw 예측값 10개
dim=1에는 각 출력의 개별값

raw 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻는다

In [5]:
X =torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

print(pred_probab)
print(f'Predicted class {y_pred}')

tensor([[0.0899, 0.0922, 0.1021, 0.0987, 0.1010, 0.1020, 0.0973, 0.1072, 0.1032,
         0.1064]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Predicted class tensor([7], device='cuda:0')


## 모델계층(Layer)

28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보자.

In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
# nn.Flatten
# nn.Flatten layer를 초기화하여 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환
# dim=0의 미니배치 차원은 유지

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())


torch.Size([3, 784])


In [8]:
# nn.Linear
# 선형계층은 저장된 w와 b를 사용하여 입력에 linear transformation을 적용하는 모듈

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
# nn.ReLU
# nonlinear activation은 모델의 입력과 출력 사이에 복잡한 mapping을 만듬
# nonlinear activation은 선형변환 후 적용하여 nonlinearity를 도입, 신경망이 다양한 현상을 학습할 수 있도록 함.

print(f'Before ReLU : {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')
print(hidden1.size())

Before ReLU : tensor([[0.0000, 0.0000, 0.2585, 0.2653, 0.0000, 0.0000, 0.0000, 0.0000, 0.2708,
         0.0842, 0.0803, 0.0000, 0.0000, 0.0000, 0.3339, 0.2014, 0.0000, 0.1402,
         0.0000, 0.1303],
        [0.1966, 0.0000, 0.2322, 0.2467, 0.1103, 0.0000, 0.3313, 0.0000, 0.3870,
         0.0000, 0.2018, 0.0000, 0.0000, 0.0333, 0.0000, 0.0393, 0.3073, 0.1930,
         0.0000, 0.2852],
        [0.1903, 0.0000, 0.4774, 0.2535, 0.0291, 0.0000, 0.4397, 0.0000, 0.4713,
         0.1079, 0.1742, 0.0000, 0.0000, 0.0000, 0.1904, 0.0581, 0.0000, 0.2996,
         0.0000, 0.4926]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.2585, 0.2653, 0.0000, 0.0000, 0.0000, 0.0000, 0.2708,
         0.0842, 0.0803, 0.0000, 0.0000, 0.0000, 0.3339, 0.2014, 0.0000, 0.1402,
         0.0000, 0.1303],
        [0.1966, 0.0000, 0.2322, 0.2467, 0.1103, 0.0000, 0.3313, 0.0000, 0.3870,
         0.0000, 0.2018, 0.0000, 0.0000, 0.0333, 0.0000, 0.0393, 0.3073, 0.1930,
         0.0000, 0.2852],
      

In [12]:
# nn.Sequential
# 순서를 갖는 모듈의 컨테이너
# 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됨.

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits)

tensor([[ 0.0150, -0.1959,  0.1142,  0.0267,  0.0263, -0.2524, -0.2711, -0.1311,
          0.0241,  0.0339],
        [ 0.0979, -0.1355,  0.1628,  0.0073,  0.0696, -0.0865, -0.2349, -0.0038,
         -0.0102,  0.1468],
        [ 0.1457, -0.0890,  0.0150,  0.0293, -0.0085, -0.2161, -0.2868, -0.0706,
          0.0233,  0.1007]], grad_fn=<AddmmBackward0>)


In [14]:
# nn.Softmax
# 신경망의 마지막 선형계층은 nn.Softmax 모듈에 전달될 logits를 반환
# logits는 모델의 각 class에 대한 예측 확률을 나타내도록 [0, 1]범위로 비례하여 조정됨.
# dim 매개변수는 값의 합이 1이 되는 차원

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1070, 0.0866, 0.1182, 0.1083, 0.1082, 0.0819, 0.0804, 0.0925, 0.1080,
         0.1090],
        [0.1094, 0.0866, 0.1167, 0.0999, 0.1063, 0.0910, 0.0784, 0.0988, 0.0982,
         0.1148],
        [0.1189, 0.0941, 0.1044, 0.1059, 0.1019, 0.0828, 0.0772, 0.0958, 0.1052,
         0.1137]], grad_fn=<SoftmaxBackward0>)


## 모델 매개변수

신경망 내부의 많은 계층들은 매개변수화(parameterize) 됨.
즉 학습중에 최적화 되는 가중치와 편향과 연관지어짐.
nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적,
모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게됨.

In [15]:
print(f'Model structure : {model} \n\n')

for name, param in model.named_parameters():
    print(f'Layer : {name} | Size : {param.size() } | Values : {param[:2]} \n ')

Model structure : NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer : linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[ 0.0153, -0.0293, -0.0047,  ...,  0.0320, -0.0015,  0.0024],
        [ 0.0353,  0.0109, -0.0090,  ..., -0.0263,  0.0278,  0.0250]],
       device='cuda:0', grad_fn=<SliceBackward0>) 
 
Layer : linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([ 0.0342, -0.0332], device='cuda:0', grad_fn=<SliceBackward0>) 
 
Layer : linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[ 0.0049, -0.0420,  0.0009,  ...,  0.0299, -0.0302, -0.0299],
        [-0.0200,  0.0035, -0.0098,  ..., -0.0169,  0.0409, -0.0190]],
       device='cuda:0', g